In [ ]:
import os
os.getcwd()

In [ ]:
os.listdir('/preprocessedData/sequences/')

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
import datetime
import pickle 
import matplotlib.pyplot as plt

# from EE_Models_v2 import Model1
# from EE_Images import Image

from tensorflow.keras.layers import LSTM, SimpleRNN, Dropout, Activation, GRU, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, concatenate

from tensorflow.keras.regularizers import l2, l1

from livelossplot.keras import PlotLossesCallback

# Different models functions

## Our proposed model

In [ ]:
def GRUx3_Densex3(name, X_train, y_train, bmi_train, X_val, y_val, bmi_val, n_batch, n_epochs, moldel_dir, image_dir, BMI=True):
    print('Creating functional api model')

    #input1 accelerometer sensor measurements 
    input_1 = Input(shape=(X_train.shape[1], X_train.shape[2]))
    hidden_1 = GRU(32, return_sequences=True, kernel_regularizer=l2(l=0.0001), recurrent_dropout=0.5)(input_1)
    hidden_2 = GRU(256, return_sequences=True, kernel_regularizer=l2(l=0.0001), recurrent_dropout=0.5)(hidden_1)
    hidden_3 = GRU(32, return_sequences=False, kernel_regularizer=l2(l=0.0001), recurrent_dropout=0.5)(hidden_2)
    hidden_4 = Dense(32, activation='relu')(hidden_3)
    
    #input2 BMI of participants 
    input_2 = Input(shape=(bmi_train.shape[1],))
    if BMI == True:
        hidden_8 = Dense(32)(input_2)

        #merge
        con = concatenate([hidden_4, hidden_8])
        x3 = Dense(32, activation='relu')(con)
    else:
        x3 = Dense(32, activation='relu')(hidden_4)

    x3 = Dropout(0.2)(x3) #another change
    x3 = Dense(16, activation='relu')(x3) #changes made
    x3 = Dropout(0.2)(x3)
    output = Dense(1, activation='linear')(x3)

    model = Model(inputs=[input_1, input_2], outputs=output)
    # plot_model(model, to_file=image_dir)

    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])
    model.summary()
    tbCallBack = TensorBoard(log_dir= moldel_dir + '/' + name, histogram_freq=0, write_graph=True, write_images=True)
    # monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=50, mode='auto', verbose=1)
    checkpointer = ModelCheckpoint(filepath= moldel_dir + '/' + name + '.hdf5', 
                                  save_best_only=True, verbose=1, mode='auto', monitor='val_loss')
    history = model.fit(x=[X_train, bmi_train], y=y_train, 
                      batch_size=n_batch, epochs=n_epochs, verbose=1, 
                      validation_data=([X_val, bmi_val], y_val), shuffle=True, 
                      callbacks=[checkpointer, tbCallBack])
    return history

## Other models tested

In [ ]:
def LSTM(name, X_train, y_train, bmi_train, X_val, y_val, bmi_val, n_batch, n_epochs, moldel_dir, image_dir, BMI=True):
    print('Creating functional api model')

    #input1 accelerometer sensor measurements 
    input_1 = Input(shape=(X_train.shape[1], X_train.shape[2]))
    hidden_1 = LSTM(32, return_sequences=True, kernel_regularizer=l2(l=0.0001), recurrent_dropout=0.5)(input_1)
    hidden_2 = LSTM(256, return_sequences=True, kernel_regularizer=l2(l=0.0001), recurrent_dropout=0.5)(hidden_1)
    hidden_3 = LSTM(32, return_sequences=False, kernel_regularizer=l2(l=0.0001), recurrent_dropout=0.5)(hidden_2)
    hidden_4 = Dense(32, activation='relu')(hidden_3)
    
    #input2 BMI of participants 
    input_2 = Input(shape=(bmi_train.shape[1],))
    if BMI == True:
        hidden_8 = Dense(32)(input_2)

        #merge
        con = concatenate([hidden_4, hidden_8])
        x3 = Dense(32, activation='relu')(con)
    else:
        x3 = Dense(32, activation='relu')(hidden_4)

    x3 = Dropout(0.2)(x3) #another change
    x3 = Dense(16, activation='relu')(x3) #changes made
    x3 = Dropout(0.2)(x3)
    output = Dense(1, activation='linear')(x3)

    model = Model(inputs=[input_1, input_2], outputs=output)
    # plot_model(model, to_file=image_dir)

    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])
    model.summary()
    tbCallBack = TensorBoard(log_dir= moldel_dir + '/' + name, histogram_freq=0, write_graph=True, write_images=True)
    # monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=50, mode='auto', verbose=1)
    checkpointer = ModelCheckpoint(filepath= moldel_dir + '/' + name + '.hdf5', 
                                  save_best_only=True, verbose=1, mode='auto', monitor='val_loss')
    history = model.fit(x=[X_train, bmi_train], y=y_train, 
                      batch_size=n_batch, epochs=n_epochs, verbose=1, 
                      validation_data=([X_val, bmi_val], y_val), shuffle=True, 
                      callbacks=[checkpointer, tbCallBack])
    return history

def simpleRNN(name, X_train, y_train, bmi_train, X_val, y_val, bmi_val, n_batch, n_epochs, moldel_dir, image_dir, BMI=True):
    print('Creating functional api model')

    #input1 accelerometer sensor measurements 
    input_1 = Input(shape=(X_train.shape[1], X_train.shape[2]))
    hidden_1 = SimpleRNN(32, return_sequences=True, kernel_regularizer=l2(l=0.0001), recurrent_dropout=0.5)(input_1)
    hidden_2 = SimpleRNN(256, return_sequences=True, kernel_regularizer=l2(l=0.0001), recurrent_dropout=0.5)(hidden_1)
    hidden_3 = SimpleRNN(32, return_sequences=False, kernel_regularizer=l2(l=0.0001), recurrent_dropout=0.5)(hidden_2)
    hidden_4 = Dense(32, activation='relu')(hidden_3)
    
    #input2 BMI of participants 
    input_2 = Input(shape=(bmi_train.shape[1],))
    if BMI == True:
        hidden_8 = Dense(32)(input_2)

        #merge
        con = concatenate([hidden_4, hidden_8])
        x3 = Dense(32, activation='relu')(con)
    else:
        x3 = Dense(32, activation='relu')(hidden_4)

    x3 = Dropout(0.2)(x3) #another change
    x3 = Dense(16, activation='relu')(x3) #changes made
    x3 = Dropout(0.2)(x3)
    output = Dense(1, activation='linear')(x3)

    model = Model(inputs=[input_1, input_2], outputs=output)
    # plot_model(model, to_file=image_dir)

    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])
    model.summary()
    tbCallBack = TensorBoard(log_dir= moldel_dir + '/' + name, histogram_freq=0, write_graph=True, write_images=True)
    # monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=50, mode='auto', verbose=1)
    checkpointer = ModelCheckpoint(filepath= moldel_dir + '/' + name + '.hdf5', 
                                  save_best_only=True, verbose=1, mode='auto', monitor='val_loss')
    history = model.fit(x=[X_train, bmi_train], y=y_train, 
                      batch_size=n_batch, epochs=n_epochs, verbose=1, 
                      validation_data=([X_val, bmi_val], y_val), shuffle=True, 
                      callbacks=[checkpointer, tbCallBack])
    return history

def GRUx5_Densex3(name, X_train, y_train, bmi_train, X_val, y_val, bmi_val, n_batch, n_epochs, moldel_dir, image_dir, BMI=True):
    print('Creating functional api model')

    #input1 accelerometer sensor measurements 
    input_1 = Input(shape=(X_train.shape[1], X_train.shape[2]))
    hidden_1 = GRU(32, return_sequences=True, kernel_regularizer=l2(l=0.0001), recurrent_dropout=0.5)(input_1)
    hidden_2_1 = GRU(32, return_sequences=True, kernel_regularizer=l2(l=0.0001), recurrent_dropout=0.5)(hidden_1)
    hidden_2_2 = GRU(256, return_sequences=True, kernel_regularizer=l2(l=0.0001), recurrent_dropout=0.5)(hidden_2_1)
    hidden_2_3 = GRU(32, return_sequences=True, kernel_regularizer=l2(l=0.0001), recurrent_dropout=0.5)(hidden_2_2)
    hidden_3 = GRU(32, return_sequences=False, kernel_regularizer=l2(l=0.0001), recurrent_dropout=0.5)(hidden_2_3)
    hidden_4 = Dense(32, activation='relu')(hidden_3)  
    
    #input2 BMI of participants 
    input_2 = Input(shape=(bmi_train.shape[1],))
    if BMI == True:
        hidden_8 = Dense(32)(input_2)

        #merge
        con = concatenate([hidden_4, hidden_8])
        x3 = Dense(32, activation='relu')(con)
    else:
        x3 = Dense(32, activation='relu')(hidden_4)

    x3 = Dropout(0.2)(x3) #another change
    x3 = Dense(16, activation='relu')(x3) #changes made
    x3 = Dropout(0.2)(x3)
    output = Dense(1, activation='linear')(x3)

    model = Model(inputs=[input_1, input_2], outputs=output)
    # plot_model(model, to_file=image_dir)

    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])
    model.summary()
    tbCallBack = TensorBoard(log_dir= moldel_dir + '/' + name, histogram_freq=0, write_graph=True, write_images=True)
    # monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=50, mode='auto', verbose=1)
    checkpointer = ModelCheckpoint(filepath= moldel_dir + '/' + name + '.hdf5', 
                                  save_best_only=True, verbose=1, mode='auto', monitor='val_loss')
    history = model.fit(x=[X_train, bmi_train], y=y_train, 
                      batch_size=n_batch, epochs=n_epochs, verbose=1, 
                      validation_data=([X_val, bmi_val], y_val), shuffle=True, 
                      callbacks=[checkpointer, tbCallBack])
    return history

def GRUx4_Densex2(name, X_train, y_train, bmi_train, X_val, y_val, bmi_val, n_batch, n_epochs, moldel_dir, image_dir, BMI=True):
    print('Creating functional api model')

    #input1 accelerometer sensor measurements 
    input_1 = Input(shape=(X_train.shape[1], X_train.shape[2]))
    hidden_1 = GRU(32, return_sequences=True, kernel_regularizer=l2(l=0.0001), recurrent_dropout=0.5)(input_1)
    hidden_2_1 = GRU(32, return_sequences=True, kernel_regularizer=l2(l=0.0001), recurrent_dropout=0.5)(hidden_1)
    hidden_2_2 = GRU(256, return_sequences=True, kernel_regularizer=l2(l=0.0001), recurrent_dropout=0.5)(hidden_2_1)
    hidden_3 = GRU(32, return_sequences=False, kernel_regularizer=l2(l=0.0001), recurrent_dropout=0.5)(hidden_2_2)
    hidden_4 = Dense(32, activation='relu')(hidden_3)  
    
    #input2 BMI of participants 
    input_2 = Input(shape=(bmi_train.shape[1],))
    if BMI == True:
        hidden_8 = Dense(32)(input_2)

        #merge
        con = concatenate([hidden_4, hidden_8])
        x3 = Dense(32, activation='relu')(con)
    else:
        x3 = Dense(32, activation='relu')(hidden_4)

    x3 = Dropout(0.2)(x3) #another change
    x3 = Dense(16, activation='relu')(x3) #changes made
    x3 = Dropout(0.2)(x3)
    output = Dense(1, activation='linear')(x3)

    model = Model(inputs=[input_1, input_2], outputs=output)
    # plot_model(model, to_file=image_dir)

    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])
    model.summary()
    tbCallBack = TensorBoard(log_dir= moldel_dir + '/' + name, histogram_freq=0, write_graph=True, write_images=True)
    # monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=50, mode='auto', verbose=1)
    checkpointer = ModelCheckpoint(filepath= moldel_dir + '/' + name + '.hdf5', 
                                  save_best_only=True, verbose=1, mode='auto', monitor='val_loss')
    history = model.fit(x=[X_train, bmi_train], y=y_train, 
                      batch_size=n_batch, epochs=n_epochs, verbose=1, 
                      validation_data=([X_val, bmi_val], y_val), shuffle=True, 
                      callbacks=[checkpointer, tbCallBack])
    return history

def GRUx3_Densex2(name, X_train, y_train, bmi_train, X_val, y_val, bmi_val, n_batch, n_epochs, moldel_dir, image_dir, BMI=True):
    print('Creating functional api model')

    #input1 accelerometer sensor measurements 
    input_1 = Input(shape=(X_train.shape[1], X_train.shape[2]))
    gru1 = GRU(32, return_sequences=True, kernel_regularizer=l2(l=0.0001), recurrent_dropout=0.5)(input_1)
    gru2 = GRU(256, return_sequences=True, kernel_regularizer=l2(l=0.0001), recurrent_dropout=0.5)(gru1)
    gru3 = GRU(32, return_sequences=False, kernel_regularizer=l2(l=0.0001), recurrent_dropout=0.5)(gru2)
    den1 = Dense(32, activation='relu')(gru3)
    
    #input2 BMI of participants 
    input_2 = Input(shape=(bmi_train.shape[1],))
    if BMI == True:
        bmi_layer = Dense(32)(input_2)

        #merge
        con = concatenate([den1, bmi_layer])
        den2 = Dense(32, activation='relu')(con)
    else:
        den2 = Dense(32, activation='relu')(den1)

    drop1 = Dropout(0.2)(den2) #another change
#     den3 = Dense(16, activation='relu')(drop1) #changes made
#     drop2 = Dropout(0.2)(den3)
    output = Dense(1, activation='linear')(drop1)

    model = Model(inputs=[input_1, input_2], outputs=output)

# plot_model(model, to_file=image_dir)

    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])
    model.summary()
    tbCallBack = TensorBoard(log_dir= moldel_dir + '/' + name, histogram_freq=0, write_graph=True, write_images=True)
    # monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=50, mode='auto', verbose=1)
    checkpointer = ModelCheckpoint(filepath= moldel_dir + '/' + name + '.hdf5', 
                                  save_best_only=True, verbose=1, mode='auto', monitor='val_loss')
    history = model.fit(x=[X_train, bmi_train], y=y_train, 
                      batch_size=n_batch, epochs=n_epochs, verbose=1, 
                      validation_data=([X_val, bmi_val], y_val), shuffle=True, 
                      callbacks=[checkpointer, tbCallBack])
    return history


def GRUx2_Densex3(name, X_train, y_train, bmi_train, X_val, y_val, bmi_val, n_batch, n_epochs, moldel_dir, image_dir, BMI=True):
    print('Creating functional api model')

    #input1 accelerometer sensor measurements 
    input_1 = Input(shape=(X_train.shape[1], X_train.shape[2]))
    gru1 = GRU(32, return_sequences=True, kernel_regularizer=l2(l=0.0001), recurrent_dropout=0.5)(input_1)
    gru2 = GRU(256, return_sequences=False, kernel_regularizer=l2(l=0.0001), recurrent_dropout=0.5)(gru1)
#     gru3 = GRU(32, return_sequences=False, kernel_regularizer=l2(l=0.0001), recurrent_dropout=0.5)(gru1)
    den1 = Dense(32, activation='relu')(gru2)
    
    #input2 BMI of participants 
    input_2 = Input(shape=(bmi_train.shape[1],))
    if BMI == True:
        bmi_layer = Dense(32)(input_2)

        #merge
        con = concatenate([den1, bmi_layer])
        den2 = Dense(32, activation='relu')(con)
    else:
        den2 = Dense(32, activation='relu')(den1)

    drop1 = Dropout(0.2)(den2) #another change
    den3 = Dense(16, activation='relu')(drop1) #changes made
    drop2 = Dropout(0.2)(den3)
    output = Dense(1, activation='linear')(den3)

    model = Model(inputs=[input_1, input_2], outputs=output)

# plot_model(model, to_file=image_dir)

    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])
    model.summary()
    tbCallBack = TensorBoard(log_dir= moldel_dir + '/' + name, histogram_freq=0, write_graph=True, write_images=True)
    # monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=50, mode='auto', verbose=1)
    checkpointer = ModelCheckpoint(filepath= moldel_dir + '/' + name + '.hdf5', 
                                  save_best_only=True, verbose=1, mode='auto', monitor='val_loss')
    history = model.fit(x=[X_train, bmi_train], y=y_train, 
                      batch_size=n_batch, epochs=n_epochs, verbose=1, 
                      validation_data=([X_val, bmi_val], y_val), shuffle=True, 
                      callbacks=[checkpointer, tbCallBack])
    return history

def GRUx2_Densex2(name, X_train, y_train, bmi_train, X_val, y_val, bmi_val, n_batch, n_epochs, moldel_dir, image_dir, BMI=True):
    print('Creating functional api model')

    #input1 accelerometer sensor measurements 
    input_1 = Input(shape=(X_train.shape[1], X_train.shape[2]))
    gru1 = GRU(32, return_sequences=True, kernel_regularizer=l2(l=0.0001), recurrent_dropout=0.5)(input_1)
    gru2 = GRU(256, return_sequences=False, kernel_regularizer=l2(l=0.0001), recurrent_dropout=0.5)(gru1)
#     gru3 = GRU(32, return_sequences=False, kernel_regularizer=l2(l=0.0001), recurrent_dropout=0.5)(gru2)
    den1 = Dense(32, activation='relu')(gru2)
    
    #input2 BMI of participants 
    input_2 = Input(shape=(bmi_train.shape[1],))
    if BMI == True:
        bmi_layer = Dense(32)(input_2)

        #merge
        con = concatenate([den1, bmi_layer])
        den2 = Dense(32, activation='relu')(con)
    else:
        den2 = Dense(32, activation='relu')(den1)

    drop1 = Dropout(0.2)(den2) #another change
#     den3 = Dense(16, activation='relu')(drop1) #changes made
#     drop2 = Dropout(0.2)(den3)
    output = Dense(1, activation='linear')(drop1)

    model = Model(inputs=[input_1, input_2], outputs=output)

# plot_model(model, to_file=image_dir)

    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])
    model.summary()
    tbCallBack = TensorBoard(log_dir= moldel_dir + '/' + name, histogram_freq=0, write_graph=True, write_images=True)
    # monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=50, mode='auto', verbose=1)
    checkpointer = ModelCheckpoint(filepath= moldel_dir + '/' + name + '.hdf5', 
                                  save_best_only=True, verbose=1, mode='auto', monitor='val_loss')
    history = model.fit(x=[X_train, bmi_train], y=y_train, 
                      batch_size=n_batch, epochs=n_epochs, verbose=1, 
                      validation_data=([X_val, bmi_val], y_val), shuffle=True, 
                      callbacks=[checkpointer, tbCallBack])
    return history

def GRUx1_Densex2(name, X_train, y_train, bmi_train, X_val, y_val, bmi_val, n_batch, n_epochs, moldel_dir, image_dir, BMI=True):
    print('Creating functional api model')

    #input1 accelerometer sensor measurements 
    input_1 = Input(shape=(X_train.shape[1], X_train.shape[2]))
#     gru1 = GRU(32, return_sequences=True, kernel_regularizer=l2(l=0.0001), recurrent_dropout=0.5)(input_1)
#     gru2 = GRU(256, return_sequences=True, kernel_regularizer=l2(l=0.0001), recurrent_dropout=0.5)(gru1)
    gru3 = GRU(32, return_sequences=False, kernel_regularizer=l2(l=0.0001), recurrent_dropout=0.5)(input_1)
    den1 = Dense(32, activation='relu')(gru3)
    
    #input2 BMI of participants 
    input_2 = Input(shape=(bmi_train.shape[1],))
    if BMI == True:
        bmi_layer = Dense(32)(input_2)

        #merge
        con = concatenate([den1, bmi_layer])
        den2 = Dense(32, activation='relu')(con)
    else:
        den2 = Dense(32, activation='relu')(den1)

    drop1 = Dropout(0.2)(den2) #another change
#     den3 = Dense(16, activation='relu')(drop1) #changes made
#     drop2 = Dropout(0.2)(den3)
    output = Dense(1, activation='linear')(drop1)

    model = Model(inputs=[input_1, input_2], outputs=output)

# plot_model(model, to_file=image_dir)

    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])
    model.summary()
    tbCallBack = TensorBoard(log_dir= moldel_dir + '/' + name, histogram_freq=0, write_graph=True, write_images=True)
    # monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=50, mode='auto', verbose=1)
    checkpointer = ModelCheckpoint(filepath= moldel_dir + '/' + name + '.hdf5', 
                                  save_best_only=True, verbose=1, mode='auto', monitor='val_loss')
    history = model.fit(x=[X_train, bmi_train], y=y_train, 
                      batch_size=n_batch, epochs=n_epochs, verbose=1, 
                      validation_data=([X_val, bmi_val], y_val), shuffle=True, 
                      callbacks=[checkpointer, tbCallBack])
    return history

In [6]:
# train models function
def train_model(list_of_participants, orig_seq_path, seq_path, device, EE_modeling, image_dir, model_dir, predPlot_dir, BMI):
######
# Train a given model using LOSO-CV given the training sequences. If you want to train with predicted activities, 
# use as input the sequences with predicted activity labels included.
    # list_of_participants: the list of participants to perform LOSO (all 28 that have COSMED data)
    # orig_seq_path: sequences directory with original EEm (for validation set)
    # seq_path: sequences directory with downsampled (10seconds) EEm (for train set)
    # device: which accelerometers to use
        # - 'aw': for both ankle and wrist data
        # - 'ankle': for only ankle data
        # - 'wrist': for only wrist data
    # EE_modeling: model function
    # image_dir: directory to save train images
    # model_dir: directory to save trained LOSO-CV models
    # predPlot_dir: directory to save trueVSpredicted plots    
    # BMI: 'True' or 'False'. True in order to train with participant level data (age, sex, weight, height, BMI),
    #                         False in order to train model without them
######        
    #initialize
    if not os.path.exists(model_dir):
        os.makedirs(model_dir) 
    if not os.path.exists(image_dir):
        os.makedirs(image_dir)
        os.makedirs(predPlot_dir)
    # run over all models
    for i in list_of_participants:
        if i in [2,3,4,19]: 
            continue
        else:
            if len(str(i)) ==1:
                name = 'GOTOV0'+str(i)
            else:
                name = 'GOTOV'+str(i)
            #fi
        #fi
        if os.path.exists(seq_path+name+'.pkl') == True:
            part =  model_name+'_'+ name
            print('Load sequences of participant', name)
            with open(seq_path+name+'.pkl','rb') as f:
                X_train, y_train, ytrain_time, bmi_train, X_val, y_val, yval_time, bmi_val, X_test, y_test, ytest_time, bmi_test, scaler = pickle.load(f)
            
            # load the orignal val and test
            with open(orig_seq_path+name+'.pkl','rb') as f:
                X_train1, y_train1, ytrain_time1, bmi_train1, X_val, y_val, yval_time, bmi_val, X_test, y_test, ytest_time, bmi_test, scaler = pickle.load(f)

            X_train1, y_train1, ytrain_time1 = [],[],[]

            # keep only one device data
            if device == 'ankle':
                X_train = X_train[:,:,0:3]
                X_val = X_val[:,:,0:3]
                X_test = X_test[:,:,0:3]
            elif device == 'wrist':
                X_train = X_train[:,:,3:6]
                X_val = X_val[:,:,3:6]
                X_test = X_test[:,:,3:6]

            print('Training model... ')
            model = EE_modeling(name, X_train, y_train, bmi_train, X_val, y_val, bmi_val, n_batch, epochs, model_dir, image_dir, BMI)

            plt.plot(model.history['mse'])
            plt.plot(model.history['val_mse'])
            plt.title('model Mean Square Error')
            plt.ylabel('Mean Square Error')
            plt.xlabel('epoch')
            plt.legend(['train', 'val'], loc='upper left')
            plt.savefig(image_dir + name + '_mse.pdf')
            plt.show()
            # summarize history for loss
            plt.plot(model.history['loss'])
            plt.plot(model.history['val_loss'])
            plt.title('model loss')
            plt.ylabel('loss')
            plt.xlabel('epoch')
            plt.legend(['train', 'val'], loc='upper left')
            plt.savefig(image_dir + name + '_loss.pdf')
            plt.show()
        else:
            print('filepath doesnt exist')        
        #fi
    #efor
#eDef

In [ ]:
#predict function
def predict_EE(list_of_models, trained_models_dir, target_seqs_dir, EEm_details, results_dir, device):
######
# Predict EEm and evaluate the LOSO-CV models.
    # list_of_models: the list of trained LOSO_CV models(all 28 probably)
    # trained_models_dir: trained models directory
    # target_seqs_dir: test sequences directory
    # results_dir: directory to save resulted figures and CSVs
    # device: which accelerometers to use
        # - 'aw': for both ankle and wrist data
        # - 'ankle': for only ankle data
        # - 'wrist': for only wrist data    
######          
    #initialize
    get_all_patients_errors = pd.DataFrame(columns=['participant', 'rsquared', 'inRsquared', 'outRsquared', 'rms', 'inRms', 'outRms'])
    stats_EEm_results = pd.DataFrame(columns=['participant', 
                                              'true_tot_mean', 'true_in_mean', 'true_out_mean', 
                                              'pred_tot_mean', 'pred_in_mean', 'pred_out_mean'])

    df_allPred = pd.DataFrame()
    df_allPredin = pd.DataFrame()
    df_allPredout = pd.DataFrame()
    
    # run over all models
    for i in list_of_models:
        if i in [2,3,4,19]: 
            continue
        else:
            if len(str(i)) ==1: 
                name = 'GOTOV0'+str(i)
            else:
                name = 'GOTOV'+str(i)
            #fi
        #fi

        # import from file the time to split indoors to outdoors data
        time = EEm_details['timeToSplit'][EEm_details['participant']==name].values[0]    
        if os.path.exists(trained_models_dir + name +'.hdf5') == True: 
            print('Predicting test for patient', name)
            with open(target_seqs_dir+name+'.pkl','rb') as f:
                X_train, y_train, ytrain_time, bmi_train,X_val, y_val, yval_time, bmi_val, X_test, y_test, ytest_time, bmi_test, scaler = pickle.load(f)

            # keep only one device data
            if device == 'ankle':
                X_train = X_train[:,:,0:3]
                X_val = X_val[:,:,0:3]
                X_test1 = X_test[:,:,0:3]
            elif device == 'wrist':
                X_train = X_train[:,:,3:6]
                X_val = X_val[:,:,3:6]
                X_test1 = X_test[:,:,3:6]
            else:
                X_test1 = X_test
            #fi
            print('loading model....')
            model = load_model(trained_models_dir+name+ '.hdf5')
            print('predicting....')
            yhat = model.predict([X_test1, bmi_test])
            
            X_test = X_test.reshape(-1, X_test.shape[2])
            
            print('inverting...')
            inv_yhat = np.empty((X_test.shape[0], 1))
            inv_yhat.fill(np.nan)
            inv_yhat[:yhat.shape[0]] = yhat

            inv_yhat = np.concatenate((X_test, inv_yhat), axis=1)
            inv_yhat = np.ma.array(inv_yhat, mask=np.isnan(inv_yhat))
            inv_yhat = scaler.inverse_transform(inv_yhat)
            inv_yhat = inv_yhat[:, 6]

            y_test = y_test.reshape(len(y_test), 1)
            inv_ytest = np.empty((X_test.shape[0], 1))
            inv_ytest.fill(np.nan)
            inv_ytest[:y_test.shape[0]] = y_test

            inv_ytest = np.concatenate((X_test, inv_ytest), axis=1)
            inv_ytest = np.ma.array(inv_ytest, mask=np.isnan(inv_ytest))
            inv_ytest = scaler.inverse_transform(inv_ytest)
            inv_ytest = inv_ytest[:, 6]

            inv_yhat = inv_yhat[~np.isnan(inv_yhat)]
            inv_ytest = inv_ytest[~np.isnan(inv_ytest)]

            df_test = pd.DataFrame(data=inv_ytest, index=ytest_time, columns=['True'])
            df_test['Predicted'] = inv_yhat

            df_test['participant'] = name        
            df_allPred = df_allPred.append(df_test)
            
            # calculate errors
            rsquared = r2_score(df_test['True'], df_test['Predicted'])
            print('rsquared...', rsquared)
            rms = sqrt(mean_squared_error(df_test['True'], df_test['Predicted']))
            print('rms...', rms)

            print('Create result CSVs and figures ...')
            true_tot_mean = np.mean(df_test['True'])
            pred_tot_mean = np.mean(df_test['Predicted'])        
            true_tot_std  = np.std(df_test['True'])
            pred_tot_std  = np.std(df_test['Predicted'])        

            z = np.polyfit(df_test['True'], df_test['Predicted'], 1)
            p = np.poly1d(z)
            x_max = int(np.max(df_test['True'])+2)
            y_max = int(np.max(df_test['Predicted'])+2)

            plt.figure(figsize=(15,8))
            plt.plot(df_test['True'], label='True_EE')
            plt.plot(df_test['Predicted'], label='Predicted_EE')
            plt.legend(loc='upper left')
            plt.title(name)
            plt.savefig(results_dir+name+'.pdf')
            # plt.show()
            plt.close()

            if np.isnan(EEm_details['outEEm'][EEm_details['participant']==name].values) == False:
                # indoors
                indoors = df_test[df_test.index <= time]#'2016-02-19 11:12:25.004000']
                indoors['participant'] = name        
                df_allPredin = df_allPredin.append(indoors)

                in_rsquared = r2_score(indoors['True'], indoors['Predicted'])
                in_rms = sqrt(mean_squared_error(indoors['True'], indoors['Predicted']))
                print('in :', in_rsquared)
                true_in_mean = np.mean(indoors['True'])
                pred_in_mean = np.mean(indoors['Predicted'])
                true_in_std  = np.std(indoors['True'])
                pred_in_std  = np.std(indoors['Predicted'])        

                # plot
                plt.figure(figsize=(15,8))
                plt.plot(indoors['True'], label='True_EE')
                plt.plot(indoors['Predicted'], label='Predicted_EE')
                plt.title(name)
                plt.legend(loc='upper left')
                plt.savefig(results_dir+name+'_in.pdf')
    #             plt.show()
                plt.close()

                # outdoors
                outdoors = df_test[df_test.index > time]#'2016-02-19 11:12:25.004000']
                outdoors['participant'] = name        
                df_allPredout = df_allPredout.append(outdoors)

                out_rsquared = r2_score(outdoors['True'], outdoors['Predicted'])
                out_rms = sqrt(mean_squared_error(outdoors['True'], outdoors['Predicted']))            
                print('out:', out_rsquared)
                true_out_mean = np.mean(outdoors['True'])
                pred_out_mean = np.mean(outdoors['Predicted'])            
                true_out_std  = np.std(outdoors['True'])
                pred_out_std  = np.std(outdoors['Predicted'])        

                # plot
                plt.figure(figsize=(15,8))
                plt.title(name)
                plt.plot(outdoors['True'], label='True_EE')
                plt.plot(outdoors['Predicted'], label='Predicted_EE')
                plt.legend(loc='upper left')
                plt.savefig(results_dir+name+'_out.pdf')
    #             plt.show()
                plt.close()

                # scatter plot
                plt.figure(figsize=(15,8))
                plt.xlim([-1,x_max])
                plt.ylim([-1,y_max])            
                plt.scatter(indoors['True'], indoors['Predicted'], c ='green', alpha=0.5, label='Indoors')
                plt.scatter(outdoors['True'], outdoors['Predicted'], c ='orange', alpha=0.5, label='Outdoors')
                plt.plot(df_test['True'],p(df_test['True']),"b")
                plt.legend(loc='upper right')
                plt.title(name)
                plt.xlabel('True')
                plt.ylabel('Predicted')
                plt.savefig(results_dir+name+'_scatter.pdf')            
    #             plt.show()
                plt.close()

            else:
                in_rsquared   = None
                in_rms        = None
                out_rsquared  = None
                out_rms       = None
                true_in_mean  = None
                true_out_mean = None
                true_in_std   = None
                true_out_std  = None
                pred_in_mean  = None
                pred_out_mean = None
                pred_in_std   = None
                pred_out_std  = None

                # scatter plot
                plt.figure(figsize=(15,8))
                plt.xlim([-1,x_max])
                plt.ylim([-1,y_max])
                plt.scatter(df_test['True'], df_test['Predicted'], c ='green', alpha=0.5, label='Indoors')
                plt.plot(df_test['True'],p(df_test['True']),"b")            
                plt.legend(loc='upper right')
                plt.title(name)
                plt.xlabel('True')
                plt.ylabel('Predicted')
                plt.savefig(results_dir+name+'_scatter.pdf')
    #             plt.show()
                plt.close()

                df_allPredin = df_allPredin.append(df_test)
            #fi

            # report results
            get_all_patients_errors = get_all_patients_errors.append({'participant': name,
                                                                      'rsquared'   : rsquared,
                                                                      'inRsquared' : in_rsquared,
                                                                      'outRsquared': out_rsquared,
                                                                      'rms'        : rms,
                                                                      'inRms'      : in_rms,
                                                                      'outRms'     : out_rms},
                                                                     ignore_index=True) 

            stats_EEm_results = stats_EEm_results.append({'participant'  : name,
                                                          'true_tot_mean': true_tot_mean,
                                                          'true_in_mean' : true_in_mean,
                                                          'true_out_mean': true_out_mean,
                                                          'true_tot_std' : true_tot_std,
                                                          'true_in_std'  : true_in_std,
                                                          'true_out_std' : true_out_std,
                                                          'pred_tot_mean': pred_tot_mean,
                                                          'pred_in_mean' : pred_in_mean,

                                                          'pred_out_mean': pred_out_mean,
                                                          'pred_tot_std' : pred_tot_std,
                                                          'pred_in_std'  : pred_in_std,
                                                          'pred_out_std' : pred_out_std},
                                                         ignore_index=True)
        else: 
            print("Model not found")            
        #fi
        print('============')
    #endFor
    
    # save results
    get_all_patients_errors.to_csv(results_dir+'errors.csv',index=False)
    stats_EEm_results.to_csv(results_dir+'predStats.csv',index=False)

    true = stats_EEm_results['true_tot_mean']
    pred = stats_EEm_results['pred_tot_mean']
    # std_true  = stats_EEm_results['true_tot_std']
    # std_pred  = stats_EEm_results['pred_tot_std']
    # scatter
    z = np.polyfit(true, pred, 1)
    p = np.poly1d(z)
    x_max = int(np.max(true)+1)
    y_max = int(np.max(pred)+1)
    plt.figure(figsize=(15,8))
    plt.xlim([1,x_max])
    plt.ylim([1,y_max])
    plt.scatter(true, pred, c ='blue', alpha=0.5)#, label='Indoors')
    plt.plot(true,p(true),"b")     
    # plt.legend(loc='upper right')
    plt.title('True Vs Predicted Average EEm per participant')
    plt.xlabel('True')
    plt.ylabel('Predicted')
    plt.show()
    plt.savefig(results_dir+'all_scatter.pdf')  
    plt.close
    
    print('######################################')
    print('Rsquared Median: '+str(get_all_patients_errors.rsquared.median())+' /SD:'+ str(get_all_patients_errors.rsquared.std()))
    print('Rsquared Mean: '+str(get_all_patients_errors.rsquared.mean())+' /rms Mean:'+ str(get_all_patients_errors.rms.mean()))
    print('--------------------------------------')
    print('inRsquared Median: '+str(get_all_patients_errors.inRsquared.median())+' /SD:'+ str(get_all_patients_errors.inRsquared.std()))
    print('inRsquared Mean: '+str(get_all_patients_errors.inRsquared.mean())+' /inRms Mean:'+ str(get_all_patients_errors.inRms.mean()))
    print('--------------------------------------')
    print('outRsquared Median: '+str(get_all_patients_errors.outRsquared.median())+' /SD:'+ str(get_all_patients_errors.outRsquared.std()))
    print('outRsquared Mean: '+str(get_all_patients_errors.outRsquared.mean())+' /outRms Mean:'+ str(get_all_patients_errors.outRms.mean()))
    print('######################################')
    
    return get_all_patients_errors, stats_EEm_results, df_allPred, df_allPredin, df_allPredout

# Train a model

In [14]:
# # # # # # # # # # # # FIXED VARs # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
n_batch = 512
epochs = 50
EEm_rate = 10
minutes = 2
aggr = 'std'
BMI = True
device = 'aw'  # aw: ankle and wrist data
               # ankle: only ankle data
               # wrist: only wrist data

model_function = 'GRUx3_dx3_EEm'
seq_path = '/preprocessedData/sequences/...seqs_1val_EEm_10ds_std_50/'
orig_seq_path = '/preprocessedData/sequences/...seqs_1val_EEm_10ds_std_50/' # test with orignal target data
model_name = model_function+'_'+str(EEm_rate)+'ds_'+str(minutes)+'mins_'+str(epochs)+'epochs_'+str(n_batch)+'batch_1val_'+aggr+'_' + device

# if you train with or without participants detail data
if BMI==False:
    model_name = model_name+'_noBMI'
else:
    model_name = model_name+'_withBMI'

model_dir = 'models/' + model_name
image_dir = 'images/' + model_name + '/Accur_Loss_figures/'
predPlot_dir = 'images/' + model_name + '/predPlot/'

print('Seqs path :', seq_path)
print('Model name:', model_name)

Seqs path : /preprocessedData/sequences/...seqs_1val_EEm_10ds_std_50/
Model name: GRUx3_dx3_EEm_10ds_2mins_50epochs_512batch_1val_std_aw_withBMI


In [ ]:
# train model
train_model(list_of_participants, orig_seq_path, seq_path, device, EE_modeling, image_dir, model_dir, predPlot_dir, BMI)

# Evaluate a model

In [15]:
# give participants to evaluate
list_of_models = list(range(5, 36+1))
# read EEm details
# EEm_details = pd.read_csv('EEm_details_per_partic.csv')  

# evaluate model with orignal target data
target_seqs_dir = orig_seq_path

trained_models_dir = model_dir+'/'

results_dir = predPlot_dir

print('Model:', trained_models_dir)
print('Seqs:', target_seqs_dir)
print('results_dir:', results_dir)

Model: models/GRUx3_dx3_EEm_10ds_2mins_50epochs_512batch_1val_std_aw_withBMI/
Seqs: /preprocessedData/sequences/...seqs_1val_EEm_10ds_std_50/
results_dir: images/GRUx3_dx3_EEm_10ds_2mins_50epochs_512batch_1val_std_aw_withBMI/predPlot/


In [ ]:
import glob
dire= list(glob.glob(trained_models_dir +'*.hdf5'))
dire.sort()
print('N=',len(dire))
dire

In [ ]:
# predict
get_all_patients_errorsf, stats_EEm_resultsf, df_allPredf, df_allPredinf, df_allPredoutf = predict_EE(list_of_models, 
                                                                                                 trained_models_dir,
                                                                                                 target_seqs_dir,
                                                                                                 EEm_details, 
                                                                                                 results_dir, 
                                                                                                 device)